In [15]:
# *****************
# IMPORT LIBRARIES
# *****************

!pip install pandas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as mtick
import os
import warnings
warnings.filterwarnings('ignore')

print("\n All Libraries are Loaded Successfully")


 All Libraries are Loaded Successfully


In [19]:
#Loading Dataset into notebook
df = pd.read_csv('/content/netflix_titles.csv')

rows = df.shape[0]
columns = df.shape[1]
print(f'The Dataset has {rows} rows and {columns} columns.')

movies = df[df['type'] == 'Movie'].shape[0]
shows = df[df['type'] == 'TV Show'].shape[0]
print(f'The Dataset has {movies} Movies and {shows} TV Shows.')

print("Here's few rows of the Netflix dataset: ")
df.head()

The Dataset has 8807 rows and 12 columns.
The Dataset has 6131 Movies and 2676 TV Shows.
Here's few rows of the Netflix dataset: 


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [21]:
#Checkout the missing values under the data and evaluate it
missing_count = df.isnull().sum()
missing_percentage = ((missing_count / len(df)) * 100).round(2)

missing_data = pd.DataFrame({
    'Missing Count' : missing_count,
    'Missing Percentage' : missing_percentage
}).sort_values('Missing Count', ascending=False)

print("\n Missing Values Under the Data")
print(missing_data[missing_data['Missing Count']>0])


 Missing Values Under the Data
            Missing Count  Missing Percentage
director             2634               29.91
country               831                9.44
cast                  825                9.37
date_added             10                0.11
rating                  4                0.05
duration                3                0.03


In [ ]:
# Handing Missing Values